In [170]:
# import necessary library
import pandas as pd
import numpy as np
import os
import glob
from fuzzywuzzy import process
import re


In [171]:
#read the data
sdata=pd.read_csv('Z:/AT Data/Tableau Database/Tableau_Short_Term.csv')
sdata=sdata.dropna(subset=['Count'])
# Convert Date column to datetime
sdata.head()

,Loc_id,District,Intersection,Year,Month,Day,Weekday_Type,Date,Hour,Mode,Direction,Count,Count_Type
0,1001,1,"4th and V St, Eureka",2014,7,Thursday,Weekday,7/31/2014,7AM - 7PM,Pedestrian,Northbound,40.0,Short term
1,1001,1,"4th and V St, Eureka",2014,8,Saturday,Weekend,8/2/2014,7AM - 7PM,Pedestrian,Northbound,54.0,Short term
2,1001,1,"4th and V St, Eureka",2014,8,Sunday,Weekend,8/3/2014,7AM - 7PM,Pedestrian,Northbound,39.0,Short term
3,1001,1,"4th and V St, Eureka",2019,8,Thursday,Weekday,8/15/2019,7AM - 7PM,Pedestrian,Northbound,27.0,Short term
4,1001,1,"4th and V St, Eureka",2019,8,Saturday,Weekend,8/17/2019,7AM - 7PM,Pedestrian,Northbound,34.0,Short term


In [172]:
# lets separate the d1 and d12 data
d1=sdata.loc[sdata['District']==1]
d1["Date"] = pd.to_datetime(d1["Date"], format="%m/%d/%Y", errors="coerce")
d1=d1.drop(['Month'],axis=1)
d1['Month'] = d1["Date"].dt.strftime("%b")
# Function to extract hours from the "Hour" column
def extract_hours(hour_range):
    # Use regex to extract time components safely
    match = re.findall(r"(\d{1,2}(?::\d{2})?[APap][Mm])", hour_range)
    
    if len(match) != 2:
        return None, None  # Return None if parsing fails

    # Convert AM/PM format to 24-hour format
    start_hour = pd.to_datetime(match[0], format="%I:%M%p" if ":" in match[0] else "%I%p").hour
    end_hour = pd.to_datetime(match[1], format="%I:%M%p" if ":" in match[1] else "%I%p").hour

    return start_hour, end_hour
# Apply function to extract start and end hours
d1[["Start_Hour", "End_Hour"]] = d1["Hour"].apply(lambda x: pd.Series(extract_hours(x)))
d1["no_hrs_collected"] = d1["End_Hour"] - d1["Start_Hour"]
d1.head()

C:\Users\s160555\AppData\Local\Temp\ipykernel_17280\2745908117.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d1["Date"] = pd.to_datetime(d1["Date"], format="%m/%d/%Y", errors="coerce")


,Loc_id,District,Intersection,Year,Day,Weekday_Type,Date,Hour,Mode,Direction,Count,Count_Type,Month,Start_Hour,End_Hour,no_hrs_collected
0,1001,1,"4th and V St, Eureka",2014,Thursday,Weekday,2014-07-31,7AM - 7PM,Pedestrian,Northbound,40.0,Short term,Jul,7,19,12
1,1001,1,"4th and V St, Eureka",2014,Saturday,Weekend,2014-08-02,7AM - 7PM,Pedestrian,Northbound,54.0,Short term,Aug,7,19,12
2,1001,1,"4th and V St, Eureka",2014,Sunday,Weekend,2014-08-03,7AM - 7PM,Pedestrian,Northbound,39.0,Short term,Aug,7,19,12
3,1001,1,"4th and V St, Eureka",2019,Thursday,Weekday,2019-08-15,7AM - 7PM,Pedestrian,Northbound,27.0,Short term,Aug,7,19,12
4,1001,1,"4th and V St, Eureka",2019,Saturday,Weekend,2019-08-17,7AM - 7PM,Pedestrian,Northbound,34.0,Short term,Aug,7,19,12


In [173]:
d1_zero=d1.loc[d1['no_hrs_collected']==0]
d1_zero

,Loc_id,District,Intersection,Year,Day,Weekday_Type,Date,Hour,Mode,Direction,Count,Count_Type,Month,Start_Hour,End_Hour,no_hrs_collected
38,1008,1,Herrick SB Offramp,2021,Thursday,Weekday,2021-05-13,12AM-12AM,Pedestrian,Northbound,15.0,Short term,May,0,0,0
39,1008,1,Herrick SB Offramp,2021,Friday,Weekday,2021-05-14,12AM-12AM,Pedestrian,Northbound,13.0,Short term,May,0,0,0
40,1008,1,Herrick SB Offramp,2021,Saturday,Weekend,2021-05-15,12AM-12AM,Pedestrian,Northbound,2.0,Short term,May,0,0,0
41,1008,1,Herrick SB Offramp,2021,Sunday,Weekend,2021-05-16,12AM-12AM,Pedestrian,Northbound,1.0,Short term,May,0,0,0
42,1009,1,Hilfiker,2021,Thursday,Weekday,2021-05-13,12AM-12AM,Pedestrian,Northbound,20.0,Short term,May,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1116,1025,1,McCullens,2021,Sunday,Weekend,2021-07-11,12AM-12AM,Bicycle,Southbound,17.0,Short term,Jul,0,0,0
1129,1028,1,Pierson's,2021,Thursday,Weekday,2021-04-29,12AM-12AM,Bicycle,Southbound,13.0,Short term,Apr,0,0,0
1130,1028,1,Pierson's,2021,Friday,Weekday,2021-04-30,12AM-12AM,Bicycle,Southbound,13.0,Short term,Apr,0,0,0
1131,1028,1,Pierson's,2021,Saturday,Weekend,2021-05-01,12AM-12AM,Bicycle,Southbound,8.0,Short term,May,0,0,0


In [174]:
d1_zero.Hour.unique()

array(['12AM-12AM'], dtype=object)

In [175]:
# it seems 12 AM to 12 AM date is wrong,, so we will replace the tme interval as 24 hours for the starting date
d1["no_hrs_collected"] = np.where(d1["no_hrs_collected"] == 0, 24, d1["no_hrs_collected"])
d1['Ave_Hourly_Volume']=(d1['Count']/d1["no_hrs_collected"]).round(2)
#d1['Hour'] = d1['Start_Hour'] + '-' + d1['End_Hour']
d1.head()

,Loc_id,District,Intersection,Year,Day,Weekday_Type,Date,Hour,Mode,Direction,Count,Count_Type,Month,Start_Hour,End_Hour,no_hrs_collected,Ave_Hourly_Volume
0,1001,1,"4th and V St, Eureka",2014,Thursday,Weekday,2014-07-31,7AM - 7PM,Pedestrian,Northbound,40.0,Short term,Jul,7,19,12,3.33
1,1001,1,"4th and V St, Eureka",2014,Saturday,Weekend,2014-08-02,7AM - 7PM,Pedestrian,Northbound,54.0,Short term,Aug,7,19,12,4.50
2,1001,1,"4th and V St, Eureka",2014,Sunday,Weekend,2014-08-03,7AM - 7PM,Pedestrian,Northbound,39.0,Short term,Aug,7,19,12,3.25
3,1001,1,"4th and V St, Eureka",2019,Thursday,Weekday,2019-08-15,7AM - 7PM,Pedestrian,Northbound,27.0,Short term,Aug,7,19,12,2.25
4,1001,1,"4th and V St, Eureka",2019,Saturday,Weekend,2019-08-17,7AM - 7PM,Pedestrian,Northbound,34.0,Short term,Aug,7,19,12,2.83


In [176]:
d1['no_hrs_collected'].describe()

count    1148.000000
mean       13.672474
std         4.157833
min        12.000000
25%        12.000000
50%        12.000000
75%        12.000000
max        24.000000
Name: no_hrs_collected, dtype: float64

In [177]:
d1['Ave_Hourly_Volume'].describe()

count    1148.000000
mean        1.179852
std         3.479411
min         0.000000
25%         0.080000
50%         0.330000
75%         1.000000
max        46.830000
Name: Ave_Hourly_Volume, dtype: float64

In [178]:
# lets separate the d12 data
d12=sdata.loc[sdata['District']==12]
# Convert the Date column to datetime
d12["Date_time"] = pd.to_datetime(d12["Date"], format="%m/%d/%Y %H:%M")
# Extract only the date part (without time)
d12["Date"] = d12["Date_time"].dt.date
# drop the datetime column
d12=d12.drop(['Date_time'],axis=1)
d12["Hour"] = pd.to_datetime(d12["Hour"], format="%I:%M %p").dt.strftime('%H:%M')
d12.head()

C:\Users\s160555\AppData\Local\Temp\ipykernel_17280\2423879986.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d12["Date_time"] = pd.to_datetime(d12["Date"], format="%m/%d/%Y %H:%M")
C:\Users\s160555\AppData\Local\Temp\ipykernel_17280\2423879986.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d12["Date"] = d12["Date_time"].dt.date


,Loc_id,District,Intersection,Year,Month,Day,Weekday_Type,Date,Hour,Mode,Direction,Count,Count_Type
1148,12001,12,PCH at 7th Ave,2022,Aug,Tuesday,Weekday,2022-08-09,23:30,Bicycle,Westbound,0.0,Short term
1149,12001,12,PCH at 7th Ave,2022,Aug,Tuesday,Weekday,2022-08-09,23:30,Bicycle,Eastbound,0.0,Short term
1150,12001,12,PCH at 7th Ave,2022,Aug,Tuesday,Weekday,2022-08-09,23:30,Bicycle,Southbound,0.0,Short term
1151,12001,12,PCH at 7th Ave,2022,Aug,Tuesday,Weekday,2022-08-09,23:30,Bicycle,Northbound,0.0,Short term
1152,12001,12,PCH at 7th Ave,2022,Aug,Tuesday,Weekday,2022-08-09,23:45,Bicycle,Westbound,0.0,Short term


In [179]:
grouped = d12.groupby(
    ['Loc_id', 'District', 'Intersection', 'Year', 'Month', 'Day', 
     'Weekday_Type', 'Date', 'Mode', 'Direction', 'Count_Type'])[['Hour']].agg(['min', 'max']).reset_index()
grouped.columns = ['Loc_id', 'District', 'Intersection', 'Year', 'Month', 'Day', 
                   'Weekday_Type', 'Date', 'Mode', 'Direction', 'Count_Type', 
                   'Start_Hour', 'End_Hour']
#grouped["End_Hour"] = np.where(grouped["End_Hour"] == '23:45', '24', grouped["End_Hour"])
grouped['Hour'] = grouped['Start_Hour'] + '-' + grouped['End_Hour']
grouped=grouped.drop(['Start_Hour','End_Hour'],axis=1)
grouped.head()

,Loc_id,District,Intersection,Year,Month,Day,Weekday_Type,Date,Mode,Direction,Count_Type,Hour
0,12001,12,PCH at 7th Ave,2022,Aug,Friday,Weekday,2022-08-12,Bicycle,Eastbound,Short term,00:00-23:45
1,12001,12,PCH at 7th Ave,2022,Aug,Friday,Weekday,2022-08-12,Bicycle,Northbound,Short term,00:00-23:45
2,12001,12,PCH at 7th Ave,2022,Aug,Friday,Weekday,2022-08-12,Bicycle,Southbound,Short term,00:00-23:45
3,12001,12,PCH at 7th Ave,2022,Aug,Friday,Weekday,2022-08-12,Bicycle,Westbound,Short term,00:00-23:45
4,12001,12,PCH at 7th Ave,2022,Aug,Friday,Weekday,2022-08-12,Pedestrian,Eastbound,Short term,00:00-23:45


In [180]:
grouped.Hour.unique()

array(['00:00-23:45', '23:30-23:45', '00:00-23:15', '00:00-22:15'],
      dtype=object)

In [181]:
grouped.dtypes

Loc_id           int64
District         int64
Intersection    object
Year             int64
Month           object
Day             object
Weekday_Type    object
Date            object
Mode            object
Direction       object
Count_Type      object
Hour            object
dtype: object

In [182]:
d12.isnull().sum()

Loc_id          0
District        0
Intersection    0
Year            0
Month           0
Day             0
Weekday_Type    0
Date            0
Hour            0
Mode            0
Direction       0
Count           0
Count_Type      0
dtype: int64

In [183]:
d12.columns

Index(['Loc_id', 'District', 'Intersection', 'Year', 'Month', 'Day',
       'Weekday_Type', 'Date', 'Hour', 'Mode', 'Direction', 'Count',
       'Count_Type'],
      dtype='object')

In [184]:
# count how many 15 mins data was collected ffor each site, each day
d12_data_count=pd.DataFrame({'no_of_data_collected':d12.groupby(['Loc_id', 'District', 'Intersection', 'Year', 'Month', 'Day',
       'Weekday_Type', 'Date', 'Mode', 'Direction','Count_Type'])['Loc_id'].count()}).reset_index()
d12_data_count['no_hrs_collected']=(d12_data_count['no_of_data_collected']/4).round(2)
# now sum the count
d12_count_sum=pd.DataFrame({'daily_count_sum':d12.groupby(['Loc_id', 'District', 'Intersection', 'Year', 'Month', 'Day',
       'Weekday_Type', 'Date', 'Mode', 'Direction','Count_Type'])['Count'].sum()}).reset_index()
# join these two dataframe together
d12_data=pd.merge(d12_data_count,d12_count_sum, on=['Loc_id', 'District', 'Intersection', 'Year', 'Month', 'Day',
       'Weekday_Type', 'Date', 'Mode', 'Direction','Count_Type'], how='inner')
d12_data1=pd.merge(d12_data,grouped, on=['Loc_id', 'District', 'Intersection', 'Year', 'Month', 'Day',
       'Weekday_Type', 'Date', 'Mode', 'Direction','Count_Type'], how='inner')
d12_data1['Ave_Hourly_Volume']=(d12_data1['daily_count_sum']/d12_data1["no_hrs_collected"]).round(2)
d12_data1.head()

,Loc_id,District,Intersection,Year,Month,Day,Weekday_Type,Date,Mode,Direction,Count_Type,no_of_data_collected,no_hrs_collected,daily_count_sum,Hour,Ave_Hourly_Volume
0,12001,12,PCH at 7th Ave,2022,Aug,Friday,Weekday,2022-08-12,Bicycle,Eastbound,Short term,96,24.0,0.0,00:00-23:45,0.00
1,12001,12,PCH at 7th Ave,2022,Aug,Friday,Weekday,2022-08-12,Bicycle,Northbound,Short term,96,24.0,36.0,00:00-23:45,1.50
2,12001,12,PCH at 7th Ave,2022,Aug,Friday,Weekday,2022-08-12,Bicycle,Southbound,Short term,96,24.0,48.0,00:00-23:45,2.00
3,12001,12,PCH at 7th Ave,2022,Aug,Friday,Weekday,2022-08-12,Bicycle,Westbound,Short term,96,24.0,1.0,00:00-23:45,0.04
4,12001,12,PCH at 7th Ave,2022,Aug,Friday,Weekday,2022-08-12,Pedestrian,Eastbound,Short term,96,24.0,311.0,00:00-23:45,12.96


In [185]:
# Function to convert 24-hour time to 12-hour format with AM/PM
def convert_to_12hr_format(hour):
    # Convert 24-hour time to 12-hour format
    return pd.to_datetime(hour, format='%H:%M').strftime('%I:%M %p')

# Apply the conversion to 'Hour' column
d12_data1[['Start_Hour_12', 'End_Hour_12']] = d12_data1['Hour'].str.split('-', expand=True)
d12_data1['Start_Hour_12'] = d12_data1['Start_Hour_12'].apply(convert_to_12hr_format)
d12_data1['End_Hour_12'] = d12_data1['End_Hour_12'].apply(convert_to_12hr_format)

# Combine the start and end hours in the desired format
d12_data1['Hour_Range'] =d12_data1['Start_Hour_12'] + ' - ' + d12_data1['End_Hour_12']
d12_data1.head()

,Loc_id,District,Intersection,Year,Month,Day,Weekday_Type,Date,Mode,Direction,Count_Type,no_of_data_collected,no_hrs_collected,daily_count_sum,Hour,Ave_Hourly_Volume,Start_Hour_12,End_Hour_12,Hour_Range
0,12001,12,PCH at 7th Ave,2022,Aug,Friday,Weekday,2022-08-12,Bicycle,Eastbound,Short term,96,24.0,0.0,00:00-23:45,0.00,12:00 AM,11:45 PM,12:00 AM - 11:45 PM
1,12001,12,PCH at 7th Ave,2022,Aug,Friday,Weekday,2022-08-12,Bicycle,Northbound,Short term,96,24.0,36.0,00:00-23:45,1.50,12:00 AM,11:45 PM,12:00 AM - 11:45 PM
2,12001,12,PCH at 7th Ave,2022,Aug,Friday,Weekday,2022-08-12,Bicycle,Southbound,Short term,96,24.0,48.0,00:00-23:45,2.00,12:00 AM,11:45 PM,12:00 AM - 11:45 PM
3,12001,12,PCH at 7th Ave,2022,Aug,Friday,Weekday,2022-08-12,Bicycle,Westbound,Short term,96,24.0,1.0,00:00-23:45,0.04,12:00 AM,11:45 PM,12:00 AM - 11:45 PM
4,12001,12,PCH at 7th Ave,2022,Aug,Friday,Weekday,2022-08-12,Pedestrian,Eastbound,Short term,96,24.0,311.0,00:00-23:45,12.96,12:00 AM,11:45 PM,12:00 AM - 11:45 PM


In [189]:
outlier=d12_data1.loc[d12_data1['Ave_Hourly_Volume']>400]
outlier

,Loc_id,District,Intersection,Year,Month,Day,Weekday_Type,Date,Mode,Direction,Count_Type,no_of_data_collected,no_hrs_collected,daily_count_sum,Hour,Ave_Hourly_Volume,Start_Hour_12,End_Hour_12,Hour_Range
389,12007,12,PCH at Main St,2022,Jul,Friday,Weekday,2022-07-22,Pedestrian,Northbound,Short term,96,24.0,11451.0,00:00-23:45,477.12,12:00 AM,11:45 PM,12:00 AM - 11:45 PM
390,12007,12,PCH at Main St,2022,Jul,Friday,Weekday,2022-07-22,Pedestrian,Southbound,Short term,96,24.0,9636.0,00:00-23:45,401.50,12:00 AM,11:45 PM,12:00 AM - 11:45 PM
405,12007,12,PCH at Main St,2022,Jul,Saturday,Weekend,2022-07-23,Pedestrian,Northbound,Short term,96,24.0,17946.0,00:00-23:45,747.75,12:00 AM,11:45 PM,12:00 AM - 11:45 PM
406,12007,12,PCH at Main St,2022,Jul,Saturday,Weekend,2022-07-23,Pedestrian,Southbound,Short term,96,24.0,12959.0,00:00-23:45,539.96,12:00 AM,11:45 PM,12:00 AM - 11:45 PM
413,12007,12,PCH at Main St,2022,Jul,Sunday,Weekend,2022-07-24,Pedestrian,Northbound,Short term,96,24.0,15078.0,00:00-23:45,628.25,12:00 AM,11:45 PM,12:00 AM - 11:45 PM
414,12007,12,PCH at Main St,2022,Jul,Sunday,Weekend,2022-07-24,Pedestrian,Southbound,Short term,96,24.0,13989.0,00:00-23:45,582.88,12:00 AM,11:45 PM,12:00 AM - 11:45 PM
437,12007,12,PCH at Main St,2022,Jul,Tuesday,Weekday,2022-07-26,Pedestrian,Northbound,Short term,96,24.0,9887.0,00:00-23:45,411.96,12:00 AM,11:45 PM,12:00 AM - 11:45 PM


In [190]:
d12_data.shape, d12_count_sum.shape, d12_data_count.shape

((888, 14), (888, 12), (888, 13))

In [191]:
d12_count['no_hrs_collected'].describe()

count    888.000000
mean      20.990991
std        7.752036
min        0.500000
25%       23.875000
50%       24.000000
75%       24.000000
max       24.000000
Name: no_hrs_collected, dtype: float64

In [193]:
# lets check zero hours data 
d12_data1['Ave_Hourly_Volume'].describe()

count    888.000000
mean      15.827849
std       59.196641
min        0.000000
25%        0.730000
50%        3.420000
75%        8.970000
max      747.750000
Name: Ave_Hourly_Volume, dtype: float64

In [194]:
d12_data.columns

Index(['Loc_id', 'District', 'Intersection', 'Year', 'Month', 'Day',
       'Weekday_Type', 'Date', 'Mode', 'Direction', 'Count_Type',
       'no_of_data_collected', 'no_hrs_collected', 'daily_count_sum'],
      dtype='object')

In [195]:
# select the columsn for final aggregation
d12_final=d12_data1[['Loc_id', 'District', 'Intersection', 'Year', 'Month', 'Day',
       'Weekday_Type', 'Date', 'Hour_Range','Mode', 'Direction', 'Count_Type',
        'no_hrs_collected', 'Ave_Hourly_Volume']]
d12_final=d12_final.rename(columns={'Hour_Range':'Hour'})
d1_final=d1[['Loc_id', 'District', 'Intersection', 'Year', 'Month', 'Day',
       'Weekday_Type', 'Date','Hour', 'Mode', 'Direction', 'Count_Type',
        'no_hrs_collected', 'Ave_Hourly_Volume']]
d1_12=pd.concat([d1_final,d12_final],axis=0)
d1_12=d1_12.rename(columns= {'Ave_Hourly_Volume': 'Count','no_hrs_collected': 'No_Hrs_Data_collected' })
# reorder
d1_12=d1_12[['Loc_id', 'District', 'Intersection', 'Year', 'Month', 'Day',
       'Weekday_Type', 'Date', 'Hour', 'Mode', 'Direction', 'Count','Count_Type',
       'No_Hrs_Data_collected',]]
d1_12.head()

,Loc_id,District,Intersection,Year,Month,Day,Weekday_Type,Date,Hour,Mode,Direction,Count,Count_Type,No_Hrs_Data_collected
0,1001,1,"4th and V St, Eureka",2014,Jul,Thursday,Weekday,2014-07-31 00:00:00,7AM - 7PM,Pedestrian,Northbound,3.33,Short term,12.0
1,1001,1,"4th and V St, Eureka",2014,Aug,Saturday,Weekend,2014-08-02 00:00:00,7AM - 7PM,Pedestrian,Northbound,4.50,Short term,12.0
2,1001,1,"4th and V St, Eureka",2014,Aug,Sunday,Weekend,2014-08-03 00:00:00,7AM - 7PM,Pedestrian,Northbound,3.25,Short term,12.0
3,1001,1,"4th and V St, Eureka",2019,Aug,Thursday,Weekday,2019-08-15 00:00:00,7AM - 7PM,Pedestrian,Northbound,2.25,Short term,12.0
4,1001,1,"4th and V St, Eureka",2019,Aug,Saturday,Weekend,2019-08-17 00:00:00,7AM - 7PM,Pedestrian,Northbound,2.83,Short term,12.0


In [196]:
d1_12.columns

Index(['Loc_id', 'District', 'Intersection', 'Year', 'Month', 'Day',
       'Weekday_Type', 'Date', 'Hour', 'Mode', 'Direction', 'Count',
       'Count_Type', 'No_Hrs_Data_collected'],
      dtype='object')

In [197]:
d1_12.shape

(2036, 14)

In [198]:
d1_12.isnull().sum()

Loc_id                   0
District                 0
Intersection             0
Year                     0
Month                    0
Day                      0
Weekday_Type             0
Date                     0
Hour                     0
Mode                     0
Direction                0
Count                    0
Count_Type               0
No_Hrs_Data_collected    0
dtype: int64

In [199]:
d1_12.to_csv('Z:/AT Data/Tableau Database/normalized_d1_12_short_term_data.csv')

In [ ]:
# end of the script